In [1]:
%load_ext dotenv
%dotenv

import os
from langchain_openai import ChatOpenAI
from langchain_together import ChatTogether

USE_CHINESE=True

In [2]:
if not USE_CHINESE:
    # Use Mixtral
    model = ChatOpenAI(
        base_url="https://api.together.xyz/v1",
        api_key=os.environ["KEY_TOGETHERAI"],
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",)
else:
    # Use Qwen
    model = ChatTogether(
        together_api_key=os.environ['KEY_TOGETHERAI'],
        model="Qwen/Qwen1.5-72B-Chat",
    )

In [3]:
%reload_ext dotenv

In [4]:
# https://python.langchain.com/v0.2/docs/tutorials/rag/
from random import randint
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings

In [5]:
# from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader

In [11]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://www.hk01.com/突發/1026971/長沙灣虐兒-保安道圖書館爆粗打女片瘋傳-警主動調查閃電拘母親?utm_source=01webshare&utm_medium=referral&utm_campaign=non_native"]
)

In [12]:
# Let's see what have been ingested
print(f"Number of documents:{len(documents)}")

Number of documents:1


In [13]:
docs = [
  doc.to_langchain_format()
  for doc in documents
]

In [14]:
if not USE_CHINESE:
    hfe = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
else:
    hfe = HuggingFaceEmbeddings(model_name="BAAI/bge-base-zh-v1.5")
    
vectorstore = Chroma.from_documents(documents=docs, embedding=hfe)

/Users/sim/venv/langchain/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/sim/venv/langchain/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_kwargs={'k': 6})
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [17]:
rag_chain.invoke("最詳細講述一下虐兒案的案情。答案必須以繁體中文作答")

Number of requested results 6 is greater than number of elements in index 1, updating n_results = 1


'長沙灣發生了一起虐兒案，案情涉及一名女子在保安道公共圖書館內對一名女童進行打罵，包括掌摑、搣耳仔和兜巴星等行為。這段影片在網上流傳後，警方主动調查並拘捕了該名35歲女子，她據稱是女童的母親，涉嫌「對所看管兒童或少年人虐待或忽略」。女童已被送往醫院進行檢查，警方呼籲市民在發現類似事件時應及時報警。'